## Capstone Project - Restaurant in Belgrade

### Introduction/Business Problem

As there so many restaraunts in Belgrade risk for opening new restaurant is getting bigger. 
That’s why using data science and location data of other restaurants we can reduce that risk and help small business owners to open new restaurants.

The main idea is to calculate the best location for opening a specific restaurant so the impact of competition is the lowest and restaurants have the best chance to profit.


### Data

Using Foursquare API we will gather all required data we need for finding the best location for the new restaurant.

The Foursquare Places API provides location-based experiences with diverse information about venues, users, photos, and check-ins. 
The API supports real-time access to places, Snap-to-Place that assigns users to specific locations, and Geo-tag. 

For our project, we will gather the name of the restaurant, address, and geographical coordinates and that data will be used for analysis.

### Analysis

Import libraries

In [1]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ------------------------------------------------------------
                       

In [2]:
import pandas as pd
import folium
import requests

Using Fourssquare API

In [34]:
request_parameters = {
    "client_id": "ID",
    "client_secret": "SECRET",
    "v": '20180605',
    "section": "restaurant",
    "near": "Belgrade",
    "radius": 5000,
    "limit": 200}

data = requests.get("https://api.foursquare.com/v2/venues/explore", params=request_parameters)

In [35]:
# Parsing data in JSON
d = data.json()["response"]
d.keys()

dict_keys(['suggestedFilters', 'geocode', 'headerLocation', 'headerFullLocation', 'headerLocationGranularity', 'totalResults', 'suggestedBounds', 'groups'])

Gathering usable data from response

In [36]:
items = d["groups"][0]["items"]

In [37]:
df_raw = []
for item in items:
    venue = item["venue"]
    categories, uid, name, location = venue["categories"], venue["id"], venue["name"], venue["location"]
    assert len(categories) == 1
    shortname = categories[0]["shortName"]
    address = location["formattedAddress"]
    if not "postalCode" in location:
        continue
    postalcode = location["postalCode"]
    lat = location["lat"]
    lng = location["lng"]
    datarow = (uid, name, shortname, address, lat, lng)
    df_raw.append(datarow)
df = pd.DataFrame(df_raw, columns=["uid", "name", "shortname", "address", "lat", "lng"])
print("Found %i restaurants" % len(df))
df.head(10)

Found 96 restaurants


,uid,name,shortname,address,lat,lng
0,5900d198f96b2c2191cb57e0,Hilton,Hotel,"[Kralja Milana 35 (Kralja Milutina), 11000 Бео...",44.804509,20.466071
1,598ee2aa2955134db1635b30,LUFF,Ice Cream,"[Prote Mateje 30 (Njegoseva), 11000 Belgrade, ...",44.803711,20.468922
2,4b62f063f964a520c4592ae3,Jugoslovensko dramsko pozorište,Theater,"[Kralja Milana 50, 11000 Београд, Србија]",44.805276,20.465168
3,4fe8a5bee4b0899a0b1f0704,Kultura,Cocktail,"[Kralja Milutina 4 (Krunska), 11000 Београд, С...",44.805905,20.468605
4,5c781d0a5bc27d0025ea8ccb,Crna ovca,Ice Cream,"[Njegoševa 45 (Smiljaniceva), 11000 Београд, С...",44.802860,20.470260
5,58614647158b9642d61f33a8,Street Pasta Bar,Italian,"[Kralja Milana 52 (Svetozara Markovića), 11040...",44.804687,20.465452
6,5380a315498e3241cb37b1a0,Salon de Thé by Small Tree,Tea Room,"[Svetog Save 12, 11000 Београд, Србија]",44.801386,20.467304
7,4cd2e8713e0c8cfa4d9f1a12,Delfi | Knjižara SKC,Bookstore,"[Kralja Milana 48 (ulaz iz Resavske), 11040 Бе...",44.806279,20.463949
8,595a88a41c0b3450c96051b3,Astal,Arcade,"[Kralja Milana 27 (Resavska), 11000 Београд, С...",44.806428,20.464426
9,4d4ab5a383eda1cd6047b5b1,Trpković,Bakery,"[Nemanjina 32, 11000 Београд, Србија]",44.803343,20.464343


In [38]:
# Belgrade center coordinates
belgrade = d["geocode"]["center"]

Map of gathered data

In [39]:
from folium import plugins

map_belgrade = folium.Map(location=[belgrade["lat"], belgrade["lng"]], zoom_start=14)

def add_markers(df):
    for (j, row) in df.iterrows():
        label = folium.Popup(row["name"], parse_html=True)
        folium.CircleMarker(
            [row["lat"], row["lng"]],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_belgrade)

add_markers(df)
hm_data = df[["lat", "lng"]].as_matrix().tolist()
map_belgrade.add_child(plugins.HeatMap(hm_data))

map_belgrade

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:19: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
